In [1]:
import lib.NotationLinear as nlin
import lib.score_comparison_lib as scl
import importlib

import music21 as m21
from pathlib import Path

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


In [2]:
score1_path = Path("test_scores/musicxml/longHaydn_a.musicxml")
score1 = m21.converter.parse(str(score1_path))
score2_path = Path("test_scores/musicxml/longHaydn_b.musicxml")
score2 = m21.converter.parse(str(score2_path))
# build ScoreTrees
score_lin1 = nlin.Score(score1)
score_lin2 = nlin.Score(score2)

In [4]:
importlib.reload(scl)
# compute the non common_subsequences for part 0
scl.non_common_subsequences([[0,0],[1,1],[2,2],[3,3]],[[1,1],[1,1],[3,3]])

[{'original': [0], 'compare_to': [1]}, {'original': [2], 'compare_to': []}]

In [7]:
%%time
# compute the non common_subsequences for part 0
part = 0
length = 35
seq1 = score_lin1.part_list[part].bar_list[:length]
seq2 = score_lin2.part_list[part].bar_list[:length]
print("About to compute LCS for sequences of length", len(seq1), len(seq2))
non_common_subsequences = scl.non_common_subsequences_of_measures(
    seq1, seq2
)
print(non_common_subsequences)

About to compute LCS for sequences of length 35 35
[{'original': [[[[('G5', 'None', False)],4,0,[],[],2218081748928,[],[], [('D5', 'None', False)],4,0,[],[],2218081749072,[],[], [('R', 'None', False)],4,0,[],[],2218081749216,[],[], [('D5', 'None', False)],4,0,['start', 'start'],[],2218081749264,[],[], [('C5', 'None', False)],4,0,['continue', 'continue'],[],2218081749456,[],[], [('B4', 'None', False)],4,0,['continue', 'continue'],[],2218081749648,[],[], [('A4', 'None', False)],4,0,['stop', 'stop'],[],2218081749840,[],[]]]], 'compare_to': [[[[('G5', 'None', False)],4,0,[],[],2218143362784,[],[], [('D5', 'None', False)],4,0,[],[],2218143361776,[],[], [('R', 'None', False)],4,0,[],[],2218143360768,[],[], [('D5', 'sharp', False)],4,0,['start', 'start'],[],2218143360720,[],[], [('C5', 'None', False)],4,0,['continue', 'continue'],[],2218143359328,[],[], [('B4', 'None', False)],4,0,['continue', 'continue'],[],2218143299568,[],[], [('A4', 'None', False)],4,0,['stop', 'stop'],[],2218143297600,[]

In [8]:
non_common_subsequences

[{'original': [[[[('G5', 'None', False)],4,0,[],[],3126091008464,[],[], [('D5', 'None', False)],4,0,[],[],3126091008608,[],[], [('R', 'None', False)],4,0,[],[],3126091008752,[],[], [('D5', 'None', False)],4,0,['start', 'start'],[],3126091008800,[],[], [('C5', 'None', False)],4,0,['continue', 'continue'],[],3126091008992,[],[], [('B4', 'None', False)],4,0,['continue', 'continue'],[],3126091009184,[],[], [('A4', 'None', False)],4,0,['stop', 'stop'],[],3126091009376,[],[]]]],
  'compare_to': [[[[('G5', 'None', False)],4,0,[],[],3126153285584,[],[], [('D5', 'None', False)],4,0,[],[],3126153223904,[],[], [('R', 'None', False)],4,0,[],[],3126153222896,[],[], [('D5', 'sharp', False)],4,0,['start', 'start'],[],3126153222848,[],[], [('C5', 'None', False)],4,0,['continue', 'continue'],[],3126153221456,[],[], [('B4', 'None', False)],4,0,['continue', 'continue'],[],3126153222176,[],[], [('A4', 'None', False)],4,0,['stop', 'stop'],[],3126153166560,[],[]]]]}]

4496846402853166584

In [13]:
import numpy as np
np.array([[1,2],[3,4]])

array([[1, 2],
       [3, 4]])

In [17]:
a = np.array([[1,2],[3,4],[5,6]])
len(a)

3

In [29]:
type(a[0][0])

numpy.int32